In [1]:
import pandas as pd
from Bio import SeqIO
from io import StringIO
import subprocess
import ray
from subprocess import Popen, PIPE

In [2]:
mafft_folder="/home/infosebi/Documents/Programmieren/MastersProject/genomes/aligned"
tree_folder="/home/infosebi/Documents/Programmieren/MastersProject/genomes/trees"

In [3]:
cluster_df = pd.read_csv('genomes/reclusters.txt', sep='\t', header=None)

In [4]:
prot_lists= []
prot_old = ''
for index, row in cluster_df.iterrows():
    prot = row[0]
    if prot != prot_old:
        if index > 0:
            prot_lists.append(prot_list)
        prot_list = []
        prot_list.append(row[1])
    else:
        prot_list.append(row[1])
    prot_old = prot    

In [5]:
seq_dict = {}
for record in SeqIO.parse("clusters.fasta", "fasta"):
        seq_dict[record.id]=record

In [6]:
cluster_list = []
for cluster in prot_lists:
    helper_list = []
    for protein in cluster:
        helper_list.append(seq_dict[protein])
    cluster_list.append(helper_list)

In [7]:
ray.init()

2024-01-16 10:41:07,112	INFO worker.py:1627 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


Python version:,3.10.8
Ray version:,2.5.1
Dashboard:,http://127.0.0.1:8265


In [8]:
@ray.remote
def maf_tree(cluster):
    if len(cluster) >= 2:
        out_handle = StringIO()
        SeqIO.write(cluster, out_handle, "fasta")
        fasta_data = out_handle.getvalue()
        mafft = subprocess.Popen(["mafft", "--auto", "--thread", "8", "-"], stdout=subprocess.PIPE, stdin=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        stdout_data, stderr_data = mafft.communicate(input=fasta_data)
        output_file = f'{mafft_folder}/{cluster[0].id}.fasta'
        with open(output_file, 'w') as output_handle:
            output_handle.write(stdout_data)
    if len(cluster) >= 3:
            subprocess.run(f"fasttree < {mafft_folder}/{cluster[0].id}.fasta >  {tree_folder}/{cluster[0].id}.tre",shell=True)

In [ ]:
futures = [maf_tree.remote(cluster) for cluster in cluster_list]
ray.get(futures)
ray.shutdown()

(maf_tree pid=183931) Total branch-length 0.509 after 0.03 sec
(maf_tree pid=183932)       0.70 seconds: Site likelihoods with rate category 1 of 20 [repeated 2x across cluster]
(maf_tree pid=183932) ML-NNI round 1: LogLk = -7146.585 NNIs 0 max delta 0.00 Time 0.70 [repeated 11x across cluster]
(maf_tree pid=183931) Switched to using 20 rate categories (CAT approximation) [repeated 5x across cluster]
(maf_tree pid=183931) Rate categories were divided by 0.711 so that average rate = 1.0 [repeated 5x across cluster]
(maf_tree pid=183931) CAT-based log-likelihoods may not be comparable across runs [repeated 5x across cluster]
(maf_tree pid=183931) Use -gamma for approximate but comparable Gamma(20) log-likelihoods [repeated 5x across cluster]
(maf_tree pid=183931) Turning off heuristics for final round of ML NNIs (converged) [repeated 5x across cluster]
(maf_tree pid=183931) Optimize all lengths: LogLk = -463.427 Time 0.11 [repeated 5x across cluster]
(maf_tree pid=183932)       1.24 seco

(maf_tree pid=183925)       0.12 seconds: Site likelihoods with rate category 1 of 20
(maf_tree pid=183925)       0.23 seconds: Site likelihoods with rate category 19 of 20
(maf_tree pid=183925)       0.37 seconds: ML Lengths 1 of 2 splits
(maf_tree pid=183931)       0.60 seconds: ML NNI round 3 of 6, 1 of 5 splits
(maf_tree pid=183925) FastTree Version 2.1.11 Double precision (No SSE3) [repeated 5x across cluster]
(maf_tree pid=183925) Alignment: standard input [repeated 5x across cluster]
(maf_tree pid=183925) Amino acid distances: BLOSUM45 Joins: balanced Support: SH-like 1000 [repeated 5x across cluster]
(maf_tree pid=183925) Search: Normal +NNI +SPR (2 rounds range 10) +ML-NNI opt-each=1 [repeated 5x across cluster]
(maf_tree pid=183925) TopHits: 1.00*sqrtN close=default refresh=0.80 [repeated 5x across cluster]
(maf_tree pid=183925) ML Model: Jones-Taylor-Thorton, CAT approximation with 20 rate categories [repeated 5x across cluster]
(maf_tree pid=183925) Initial topology in 0.01

(maf_tree pid=183925)       0.32 seconds: ML NNI round 1 of 5, 1 of 3 splits
(maf_tree pid=183925)       1.47 seconds: ML NNI round 2 of 5, 1 of 3 splits
(maf_tree pid=183925)       1.60 seconds: ML NNI round 3 of 5, 1 of 3 splits
(maf_tree pid=183925)       1.36 seconds: Site likelihoods with rate category 17 of 20 [repeated 5x across cluster]
(maf_tree pid=183925)       2.99 seconds: ML Lengths 1 of 3 splits
(maf_tree pid=183926)       0.32 seconds: ME NNI round 1 of 20, 1 of 28 splits
(maf_tree pid=183927) FastTree Version 2.1.11 Double precision (No SSE3) [repeated 10x across cluster]
(maf_tree pid=183927) Alignment: standard input [repeated 10x across cluster]
(maf_tree pid=183927) Amino acid distances: BLOSUM45 Joins: balanced Support: SH-like 1000 [repeated 10x across cluster]
(maf_tree pid=183927) Search: Normal +NNI +SPR (2 rounds range 10) +ML-NNI opt-each=1 [repeated 10x across cluster]
(maf_tree pid=183927) TopHits: 1.00*sqrtN close=default refresh=0.80 [repeated 10x across

(maf_tree pid=183929) Turning off heuristics for final round of ML NNIs [repeated 5x across cluster]
(maf_tree pid=183931) ML-NNI round 3: LogLk = -169.648 NNIs 0 max delta 0.00 Time 0.01 (final) [repeated 7x across cluster]
(maf_tree pid=183925)       1.05 seconds: ML NNI round 3 of 6, 1 of 6 splits [repeated 2x across cluster]
(maf_tree pid=183927)       0.10 seconds: Site likelihoods with rate category 20 of 20 [repeated 2x across cluster]
(maf_tree pid=183925) FastTree Version 2.1.11 Double precision (No SSE3) [repeated 20x across cluster]
(maf_tree pid=183925) Alignment: standard input [repeated 20x across cluster]
(maf_tree pid=183925) Amino acid distances: BLOSUM45 Joins: balanced Support: SH-like 1000 [repeated 20x across cluster]
(maf_tree pid=183925) Search: Normal +NNI +SPR (2 rounds range 10) +ML-NNI opt-each=1 [repeated 20x across cluster]
(maf_tree pid=183925) TopHits: 1.00*sqrtN close=default refresh=0.80 [repeated 20x across cluster]
(maf_tree pid=183925) ML Model: Jone

(maf_tree pid=183930)       1.56 seconds: ML Lengths 1 of 3 splits
(maf_tree pid=183925) Turning off heuristics for final round of ML NNIs [repeated 2x across cluster]
(maf_tree pid=183930) ML-NNI round 3: LogLk = -1694.990 NNIs 0 max delta 0.00 Time 1.56 (final) [repeated 4x across cluster]
(maf_tree pid=183930)       1.28 seconds: ML NNI round 3 of 5, 1 of 3 splits [repeated 2x across cluster]
(maf_tree pid=183930)       0.85 seconds: Site likelihoods with rate category 15 of 20 [repeated 3x across cluster]
(maf_tree pid=183927)       0.13 seconds: ME NNI round 1 of 9, 1 of 3 splits
(maf_tree pid=183932) FastTree Version 2.1.11 Double precision (No SSE3) [repeated 7x across cluster]
(maf_tree pid=183932) Alignment: standard input [repeated 7x across cluster]
(maf_tree pid=183932) Amino acid distances: BLOSUM45 Joins: balanced Support: SH-like 1000 [repeated 7x across cluster]
(maf_tree pid=183932) Search: Normal +NNI +SPR (2 rounds range 10) +ML-NNI opt-each=1 [repeated 7x across clu

...

(maf_tree pid=183925) Total time: 0.14 seconds Unique: 2/10 Bad splits: 0/0 [repeated 9x across cluster]
(maf_tree pid=183925) ML-NNI round 2: LogLk = -1343.086 NNIs 0 max delta 0.00 Time 0.09 [repeated 14x across cluster]
(maf_tree pid=183925) Switched to using 20 rate categories (CAT approximation) [repeated 8x across cluster]
(maf_tree pid=183925) Rate categories were divided by 0.624 so that average rate = 1.0 [repeated 8x across cluster]
(maf_tree pid=183925) CAT-based log-likelihoods may not be comparable across runs [repeated 8x across cluster]
(maf_tree pid=183925) Use -gamma for approximate but comparable Gamma(20) log-likelihoods [repeated 8x across cluster]
(maf_tree pid=183925) Turning off heuristics for final round of ML NNIs (converged) [repeated 8x across cluster]
(maf_tree pid=183929)       0.14 seconds: ML NNI round 1 of 5, 1 of 3 splits [repeated 8x across cluster]
(maf_tree pid=183928)       1.45 seconds: Site likelihoods with rate category 16 of 20 [repeated 13x acr

(maf_tree pid=183932) Total time: 0.00 seconds Unique: 1/10 Bad splits: 0/0 [repeated 14x across cluster]
(maf_tree pid=183927) ML-NNI round 1: LogLk = -408.554 NNIs 0 max delta 0.00 Time 0.01 [repeated 23x across cluster]
(maf_tree pid=183928) Switched to using 20 rate categories (CAT approximation) [repeated 11x across cluster]
(maf_tree pid=183928) Rate categories were divided by 0.629 so that average rate = 1.0 [repeated 11x across cluster]
(maf_tree pid=183928) CAT-based log-likelihoods may not be comparable across runs [repeated 11x across cluster]
(maf_tree pid=183928) Use -gamma for approximate but comparable Gamma(20) log-likelihoods [repeated 11x across cluster]
(maf_tree pid=183928) Turning off heuristics for final round of ML NNIs (converged) [repeated 11x across cluster]
(maf_tree pid=183931)       0.89 seconds: ML NNI round 3 of 6, 1 of 5 splits [repeated 8x across cluster]
(maf_tree pid=183931)       0.78 seconds: Site likelihoods with rate category 19 of 20 [repeated 14

(maf_tree pid=183926)       0.48 seconds: ME NNI round 7 of 18, 1 of 19 splits
(maf_tree pid=183931) Total time: 0.08 seconds Unique: 3/5 Bad splits: 0/0 [repeated 6x across cluster]
(maf_tree pid=183931) ML-NNI round 2: LogLk = -910.542 NNIs 0 max delta 0.00 Time 0.05 [repeated 10x across cluster]
(maf_tree pid=183931) Switched to using 20 rate categories (CAT approximation) [repeated 5x across cluster]
(maf_tree pid=183931) Rate categories were divided by 0.733 so that average rate = 1.0 [repeated 5x across cluster]
(maf_tree pid=183931) CAT-based log-likelihoods may not be comparable across runs [repeated 5x across cluster]
(maf_tree pid=183931) Use -gamma for approximate but comparable Gamma(20) log-likelihoods [repeated 5x across cluster]
(maf_tree pid=183931) Turning off heuristics for final round of ML NNIs (converged) [repeated 5x across cluster]
(maf_tree pid=183928)       0.94 seconds: ML NNI round 3 of 6, 1 of 6 splits [repeated 2x across cluster]
(maf_tree pid=183928)      

(maf_tree pid=183928)       0.23 seconds: Site likelihoods with rate category 12 of 20
(maf_tree pid=183932) FastTree Version 2.1.11 Double precision (No SSE3)
(maf_tree pid=183932) Alignment: standard input
(maf_tree pid=183932) Amino acid distances: BLOSUM45 Joins: balanced Support: SH-like 1000
(maf_tree pid=183932) Search: Normal +NNI +SPR (2 rounds range 10) +ML-NNI opt-each=1
(maf_tree pid=183932) TopHits: 1.00*sqrtN close=default refresh=0.80
(maf_tree pid=183932) ML Model: Jones-Taylor-Thorton, CAT approximation with 20 rate categories
(maf_tree pid=183932) Initial topology in 0.00 seconds
(maf_tree pid=183932) Refining topology: 6 rounds ME-NNIs, 2 rounds ME-SPRs, 3 rounds ML-NNIs
(maf_tree pid=183932) Total branch-length 0.642 after 0.00 sec
(maf_tree pid=183925) Turning off heuristics for final round of ML NNIs
(maf_tree pid=183925) Switched to using 20 rate categories (CAT approximation) [repeated 3x across cluster]
(maf_tree pid=183925) Rate categories were divided by 0.62

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [ ]:
#create zip files from genome
#include mafft_alignments
#include tree folder